# Pycode Generator

In [1]:
# Built-in library
import re
import json
from typing import Any, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
from pathlib import Path
import pandas as pd
import polars as pl
from pprint import pprint
from rich import print
import torch

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

from langchain.llms import OpenAI

### Using A Basic OpenAI Langchain LLM

- Without using a chain.
- Very simplistic.

<br>

[![image.png](https://i.postimg.cc/cLyVwqrs/image.png)](https://postimg.cc/Mv5PJF74)


In [3]:
_ = load_dotenv(find_dotenv())  # read local .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
# Create LLM
llm: OpenAI = OpenAI(openai_api_key=OPENAI_API_KEY)
prompt: str = "Write a very short poem"
result = llm(prompt=prompt)

print(result)

A sky of blue,
A sun so bright,
A day full of joy,
A delight.

## Updated Logic

- Add Chains (from langchain)

<br>

[![image.png](https://i.postimg.cc/RVLp2WgH/image.png)](https://postimg.cc/0brCwyRk)

In [5]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

### LLM Chain

[![image.png](https://i.postimg.cc/GpFWKb4L/image.png)](https://postimg.cc/1gz75h02)

In [6]:
# Used for text completion LLMs
prompt_template: str = """Write a very short {language} function that will {task}"""
code_prompt: PromptTemplate = PromptTemplate(
    input_variables=["language", "task"], template=prompt_template
)
code_chain: LLMChain = LLMChain(llm=llm, prompt=code_prompt)

# Hardcoded inputs!
language, task = ("Python", "generate a list of numbers")

result = code_chain(inputs={"language": language, "task": task})
print(result)

{
    'language': 'Python',
    'task': 'generate a list of numbers',
    'text': '\n\ndef generate_list():\n  my_list = []\n  for x in range(10):\n    my_list.append(x)\n  return 
my_list\n\nprint(generate_list())'
}

In [7]:
print(result.get("text"))

def generate_list():
  my_list = []
  for x in range(10):
    my_list.append(x)
  return my_list

print(generate_list())

## Connecting Chains Together

- Chains in series (sequential chain)

<br>

[![image.png](https://i.postimg.cc/8Cdgm9wV/image.png)](https://postimg.cc/GBpfcMZM)

In [11]:
from langchain.chains import SequentialChain

# Used for text completion LLMs
code_prompt_template: str = (
    """Write a very short {language} function that will {task}"""
)
unittest_prompt_template: str = (
    """Write a test for the following {language} code: \n{code}"""
)

# Propmt(s)
code_prompt: PromptTemplate = PromptTemplate(
    input_variables=["language", "task"], template=code_prompt_template
)
unittest_prompt: PromptTemplate = PromptTemplate(
    input_variables=["language", "code"], template=unittest_prompt_template
)

# Chain(s)
code_chain: LLMChain = LLMChain(
    llm=llm,
    prompt=code_prompt,
    output_key="code",  # from text to code
)
unittest_chain: LLMChain = LLMChain(
    llm=llm,
    prompt=unittest_prompt,
    output_key="test",  # from text to code
)
# Feed the output of code_chain into unittest_chain
final_chain: SequentialChain = SequentialChain(
    chains=[code_chain, unittest_chain],
    input_variables=["language", "task"],
    output_variables=["code", "test"],
)
# Hardcoded inputs!
language, task = ("Python", "generate a list of numbers")

result = final_chain(inputs={"language": language, "task": task})
print(result)

{
    'language': 'Python',
    'task': 'generate a list of numbers',
    'code': '\n\ndef generate_list(n): \n  num_list = [] \n  for i in range(n): \n    num_list.append(i) \n  return
num_list \n  \nprint(generate_list(5))',
    'test': "\n\nimport unittest\n\nclass TestGenerateList(unittest.TestCase):\n    def test_generate_list(self):\n
self.assertEqual(generate_list(5), [0, 1, 2, 3, 4])\n\nif __name__ == '__main__':\n    unittest.main()"
}

In [12]:
# Output of the unittest_chain (key="test")
print(result.get("test"))

import unittest

class TestGenerateList(unittest.TestCase):
    def test_generate_list(self):
        self.assertEqual(generate_list(5), [0, 1, 2, 3, 4])

if __name__ == '__main__':
    unittest.main()